In [1]:
# import nltk
# nltk.download('punkt_tab')
# nltk.download('averaged_perceptron_tagger_eng')
# nltk.download('wordnet')

In [1]:
import pandas as pd
import random
from topic_modeling_pipeline import *
from classification_pipeline import *
random.seed(42)

## AG News Dataset

In [2]:
ag_test = pd.read_csv('../data/AG News/test.csv')
ag_train = pd.read_csv('../data/AG News/train.csv')

# Define the mapping
class_mapping = {1: "World", 2: "Sports", 3: "Business", 4: "Science"}

# Apply the mapping to the class column
ag_test['Class'] = ag_test['Class Index'].replace(class_mapping)
ag_train['Class'] = ag_train['Class Index'].replace(class_mapping)

ag_news_train_baseline = ag_train['Description']
ag_news_train_true_labels = ag_train['Class Index']

sampled = ag_news_train_baseline.sample(1000, random_state=42)
ag_news_train_baseline = sampled
ag_news_train_true_labels = ag_news_train_true_labels.loc[sampled.index]

sampled_indices = ag_test.sample(3600, random_state=42).index
ag_news_baseline = ag_test.loc[sampled_indices, 'Description']
ag_news_true_labels = ag_test.loc[sampled_indices, 'Class Index']

ag_news_train = {
    "AG News":(ag_news_train_baseline, ag_news_train_true_labels),
}

ag_news = {
    "AG News":(ag_news_baseline, ag_news_true_labels),
}

In [3]:
ag_news_train_true_labels.value_counts()

Class Index
2    277
4    261
1    236
3    226
Name: count, dtype: int64

In [4]:
topic_modelling_orchestrator = TopicModelingPipelineOrchestrator()

topic_modelling_orchestrator.add_models_grid(
    model_types=['LDA', 'LSI', 'NMF'],
    param_grid={'n_topics': [4]}
)

['LDA_4', 'LSI_4', 'NMF_4']

In [5]:
class_orchestrator = ClassificationPipelineOrchestrator()

class_orchestrator.add_models_grid(
    model_types=[
        'SVMRoberta', 
        'SVM'],
    # param_grid={'SVMRoberta': [{}], 'SVM': [{}]}  # Empty dictionary means default parameters

)

['SVMRoberta', 'SVM']

In [6]:
from noise_strategy import *
noise_strategies = [
    NoNoise(),
    # AddRandomCharsNoise(), 
    # AddRandomWordsNoise(), 
    # DeleteRandomWordsNoise(), 
    # ShuffleSentencesNoise(noise_level=0.7), 
    # ReplaceWithSynonymsNoise(), 
    # ReplaceWithAntonymsNoise()
]

In [7]:
class_orchestrator.evaluate_with_training(ag_news_train, ag_news, noise_strategies)

Datasets:   0%|          | 0/1 [00:00<?, ?it/s]

Models:   0%|          | 0/2 [00:00<?, ?it/s]

Noise Strategies:   0%|          | 0/1 [00:00<?, ?it/s]

Noise Strategies:   0%|          | 0/1 [00:00<?, ?it/s]

,Dataset,Noise,Model,Accuracy,F1 Score,Precision,Recall
1,AG News,NoNoise,SVM,0.797222,0.799752,0.817493,0.797222
0,AG News,NoNoise,SVMRoberta,0.893889,0.894115,0.894916,0.893889


In [9]:
class_orchestrator.results

,Dataset,Noise,Model,Accuracy,F1 Score,Precision,Recall
1,AG News,NoNoise,SVM,0.797222,0.799752,0.817493,0.797222
0,AG News,NoNoise,SVMRoberta,0.893889,0.894115,0.894916,0.893889


In [8]:
topic_modelling_orchestrator.evaluate(ag_news, noise_strategies)

Datasets:   0%|          | 0/1 [00:00<?, ?it/s]

Models:   0%|          | 0/3 [00:00<?, ?it/s]

Noise Strategies:   0%|          | 0/1 [00:00<?, ?it/s]

2025-05-18 16:08:47,201 [INFO] gensim.corpora.dictionary: adding document #0 to Dictionary<0 unique tokens: []>
2025-05-18 16:08:47,226 [INFO] gensim.corpora.dictionary: built Dictionary<10608 unique tokens: ['citi', 'cup', 'entertain', 'epic', 'fa']...> from 3600 documents (total 65950 corpus positions)
2025-05-18 16:08:47,226 [INFO] gensim.utils: Dictionary lifecycle event {'msg': "built Dictionary<10608 unique tokens: ['citi', 'cup', 'entertain', 'epic', 'fa']...> from 3600 documents (total 65950 corpus positions)", 'datetime': '2025-05-18T16:08:47.226905', 'gensim': '4.3.0', 'python': '3.11.7 (main, Dec 15 2023, 12:09:56) [Clang 14.0.6 ]', 'platform': 'macOS-15.4.1-arm64-arm-64bit', 'event': 'created'}
2025-05-18 16:08:47,231 [INFO] gensim.topic_coherence.probability_estimation: using ParallelWordOccurrenceAccumulator<processes=10, batch_size=64> to estimate probabilities from sliding windows
2025-05-18 16:08:53,219 [INFO] gensim.topic_coherence.text_analysis: 10 accumulators retri

Noise Strategies:   0%|          | 0/1 [00:00<?, ?it/s]

2025-05-18 16:08:54,174 [INFO] gensim.corpora.dictionary: adding document #0 to Dictionary<0 unique tokens: []>
2025-05-18 16:08:54,288 [INFO] gensim.corpora.dictionary: built Dictionary<10608 unique tokens: ['citi', 'cup', 'entertain', 'epic', 'fa']...> from 3600 documents (total 65950 corpus positions)
2025-05-18 16:08:54,300 [INFO] gensim.utils: Dictionary lifecycle event {'msg': "built Dictionary<10608 unique tokens: ['citi', 'cup', 'entertain', 'epic', 'fa']...> from 3600 documents (total 65950 corpus positions)", 'datetime': '2025-05-18T16:08:54.300347', 'gensim': '4.3.0', 'python': '3.11.7 (main, Dec 15 2023, 12:09:56) [Clang 14.0.6 ]', 'platform': 'macOS-15.4.1-arm64-arm-64bit', 'event': 'created'}
2025-05-18 16:08:54,307 [INFO] gensim.topic_coherence.probability_estimation: using ParallelWordOccurrenceAccumulator<processes=10, batch_size=64> to estimate probabilities from sliding windows
2025-05-18 16:09:00,236 [INFO] gensim.topic_coherence.text_analysis: 10 accumulators retri

Noise Strategies:   0%|          | 0/1 [00:00<?, ?it/s]

2025-05-18 16:09:01,291 [INFO] gensim.corpora.dictionary: adding document #0 to Dictionary<0 unique tokens: []>
2025-05-18 16:09:01,346 [INFO] gensim.corpora.dictionary: built Dictionary<10608 unique tokens: ['citi', 'cup', 'entertain', 'epic', 'fa']...> from 3600 documents (total 65950 corpus positions)
2025-05-18 16:09:01,365 [INFO] gensim.utils: Dictionary lifecycle event {'msg': "built Dictionary<10608 unique tokens: ['citi', 'cup', 'entertain', 'epic', 'fa']...> from 3600 documents (total 65950 corpus positions)", 'datetime': '2025-05-18T16:09:01.365577', 'gensim': '4.3.0', 'python': '3.11.7 (main, Dec 15 2023, 12:09:56) [Clang 14.0.6 ]', 'platform': 'macOS-15.4.1-arm64-arm-64bit', 'event': 'created'}
2025-05-18 16:09:01,369 [INFO] gensim.topic_coherence.probability_estimation: using ParallelWordOccurrenceAccumulator<processes=10, batch_size=64> to estimate probabilities from sliding windows
2025-05-18 16:09:07,614 [INFO] gensim.topic_coherence.text_analysis: 10 accumulators retri

In [8]:
topic_modelling_orchestrator.results

{}